# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('../WeatherPy/output_data/weather_data.csv')
weather_df.head()

,Unnamed: 0,City,Latitude,Longitude,Temperature °F,Humidity (%),Cloud Coverage (%),Wind Speed (MPH)
0,0,Paamiut,59.702501,-52.265104,25.92,86.0,60.0,8.23
1,1,Bandarbeyla,6.657520,58.129427,73.60,74.0,85.0,13.27
2,2,Nizhneyansk,89.893231,133.970288,NaN,NaN,NaN,NaN
3,3,Rikitea,-34.638351,-113.096852,79.09,71.0,82.0,8.52
4,4,Mehamn,78.996117,27.497730,26.60,86.0,0.0,12.66


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
gmaps.configure(api_key=gkey)
fig = gmaps.figure()
max_humid = weather_df['Humidity (%)'].max()

heat_layer = gmaps.heatmap_layer(weather_df[['Latitude', 'Longitude']], weights= weather_df['Humidity (%)'], 
                                 dissipating=False, max_intensity=max_humid,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
ideal_df = weather_df.loc[(weather_df['Temperature \N{DEGREE SIGN}F'] > 0) & 
                          (weather_df['Temperature \N{DEGREE SIGN}F'] < 30) &
                          (weather_df['Humidity (%)'] < 90) &
                          (weather_df['Cloud Coverage (%)'] == 0)
                         ]
print(ideal_df.shape)
ideal_df

(10, 8)


,Unnamed: 0,City,Latitude,Longitude,Temperature °F,Humidity (%),Cloud Coverage (%),Wind Speed (MPH)
4,4,Mehamn,78.996117,27.497730,26.60,86.0,0.0,12.66
16,16,Vanino,49.432710,140.558586,6.39,89.0,0.0,3.27
32,32,Qaanaaq,86.323867,-77.249508,7.66,69.0,0.0,8.84
171,171,Husavik,80.591382,-6.553225,26.60,86.0,0.0,3.44
243,243,Lasa,33.733615,91.476288,23.70,79.0,0.0,5.75
252,252,Evensk,61.034066,160.346849,0.34,83.0,0.0,7.16
309,309,Hohhot,41.625677,110.586836,19.40,58.0,0.0,6.71
424,424,Horsham,-36.683393,141.540254,29.21,74.0,0.0,9.22
429,429,Kjollefjord,70.046718,27.267673,26.60,86.0,0.0,12.66
588,588,Zlobin,52.648052,29.831754,29.95,47.0,0.0,6.91


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
ideal_df['Hotel Name'] = ""
ideal_df['Country'] = ""
ideal_df = ideal_df.reset_index()
ideal_df.head()

C:\Users\Owner\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Owner\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,index,Unnamed: 0,City,Latitude,Longitude,Temperature °F,Humidity (%),Cloud Coverage (%),Wind Speed (MPH),Hotel Name,Country
0,4,4,Mehamn,78.996117,27.497730,26.60,86.0,0.0,12.66,,
1,16,16,Vanino,49.432710,140.558586,6.39,89.0,0.0,3.27,,
2,32,32,Qaanaaq,86.323867,-77.249508,7.66,69.0,0.0,8.84,,
3,171,171,Husavik,80.591382,-6.553225,26.60,86.0,0.0,3.44,,
4,243,243,Lasa,33.733615,91.476288,23.70,79.0,0.0,5.75,,


In [6]:
# geocoordinates

target_radius = 10000 * 1000
target_type = "lodging"
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotels = []
countries = []

for index, row in ideal_df.iterrows():
    
    target_coordinates = f'{row["Latitude"]},{row["Longitude"]}'

    # set up a parameters dictionary
    params = {
    "location": target_coordinates,    
    "type": target_type,   
    "radius": target_radius,
    "key": gkey   
    }
    
    print(target_coordinates)
    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()

    try:
        hotel_name = response['results'][0]['name']
        country_response = response['results'][0]['plus_code']['compound_code']
        country_name = country_response.split(", ",1)[1]
        
        hotels.append(hotel_name)
        countries.append(country_name)
        
    except:
        
        print(f'No results for #{index}: {row["City"]}. Skipping...')
        hotels.append("")
        countries.append("")
        

78.9961174208056,27.497730343488197
49.4327099806886,140.55858645700866
No results for #1: Vanino. Skipping...
86.32386737362732,-77.24950750361812
80.59138183483742,-6.553225081301804
33.73361466684882,91.4762883099063
No results for #4: Lasa. Skipping...
61.03406594369844,160.34684895490562
No results for #5: Evensk. Skipping...
41.625676600687314,110.58683638766442
No results for #6: Hohhot. Skipping...
-36.68339348823001,141.54025399219418
No results for #7: Horsham. Skipping...
70.04671757132277,27.267673068011387
52.648051989578114,29.831753744752287
No results for #9: Zlobin. Skipping...


In [7]:
ideal_df['Hotel Name'] = hotels
ideal_df['Country'] = countries
ideal_df = ideal_df.loc[ideal_df['Hotel Name'] != ""]
ideal_df

,index,Unnamed: 0,City,Latitude,Longitude,Temperature °F,Humidity (%),Cloud Coverage (%),Wind Speed (MPH),Hotel Name,Country
0,4,4,Mehamn,78.996117,27.497730,26.60,86.0,0.0,12.66,Primus Valencia,Spain
2,32,32,Qaanaaq,86.323867,-77.249508,7.66,69.0,0.0,8.84,Appart'City Pau Centre - Appart Hôtel,France
3,171,171,Husavik,80.591382,-6.553225,26.60,86.0,0.0,3.44,Hotel Alcocebre Suites,Spain
8,429,429,Kjollefjord,70.046718,27.267673,26.60,86.0,0.0,12.66,فندق النخيل,Algeria


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_df.iterrows()]
locations = ideal_df[["Latitude", "Longitude"]]

In [12]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='red',
    stroke_color='red', 
    info_box_content=hotel_info
)


fig.add_layer(hotel_layer)

fig

# Display figure


Figure(layout=FigureLayout(height='420px'))